---
# **QuickStart**
---
## CSRB Node
* Start CSRB by running Steps #A1-A3.

## CSRBappNotHotdog Server
* Start CSRB by running Steps #A1-A3.
* Start CSRBappNothostdog Server by running Steps #B1 and #B2.

## CSRBappNotHotdog Client
* Start CSRB by running Steps #A1-A3.
* Identify the NODEID of the CSRBappNotHostdog Server that will be used for the processing (see output of Step A1 of Server's Notebook).
* Set the SERVERID in Step #B3 with the Server's NODEID.
* Select one of the available Images or upload/set your own, in Step #B3.
* Run Step #B3 and wait for the image recognition outcome.

# NOTES
* To run the CSRBnode Notebook in multiple instances, save a copy in Drive.

<br>

---
---

# CSRB INIT

In [ ]:
#@title Show NODEID (Step #A1)

import os
import hashlib

#print(os.environ["HOSTNAME"])
NODEID = hashlib.md5(os.environ["HOSTNAME"].encode('utf-8')).hexdigest().upper()
print("NODEID: " + NODEID)

In [ ]:
#@title Clone CSRBnotebook (Step #A2)
!git -C /content/ clone --recurse-submodules git://github.com/CSRBapp/CSRBnotebook.git

In [ ]:
#@title Update CSRBnotebook
!git -C /content/CSRBnotebook pull --recurse-submodules

# CSRB RUN


In [ ]:
#@title Start CSRBvfsFUSE (Step #A3)

import os
import hashlib
import psutil

assert "CSRBvfsFUSE" not in (p.name() for p in psutil.process_iter()), ("CSRBvfsFUSE is already running")

#print(os.environ["HOSTNAME"])
NODEID = hashlib.md5(os.environ["HOSTNAME"].encode('utf-8')).hexdigest().upper()
print("NODEID: " + NODEID)

ROUTERHOST = "public.csrb.app" #@param {type: "string"}
ROUTERPORT = "32450" #@param {type: "string"}
ROUTERINTERSPACEUSEC = "300000" #@param {type: "string"}
CSRBNOTEBOOKROOT = "/content/CSRBnotebook/"
global CSRBVFS
CSRBVFS = "/mnt/CSRB" #@param {type: "string"}
RUNBG = True #@param {type: "boolean"}

print("Starting CSRBvfsFUSE " + ("in Background" if RUNBG is True else "in Foreground"))

!chmod a+x {CSRBNOTEBOOKROOT}/CSRBbin/UBUNTU-18.04/CSRBvfsFUSE
!mkdir {CSRBVFS} 2> /dev/null
!umount -f {CSRBVFS} 2> /dev/null

os.system(CSRBNOTEBOOKROOT + "/CSRBbin/UBUNTU-18.04/CSRBvfsFUSE -o " \
"nodev,allow_other,bindHost=0.0.0.0,bindPort=0," \
"routerHost=" + ROUTERHOST + ",routerPort=" + ROUTERPORT + "," \
"routerInterspaceUSEC=" + ROUTERINTERSPACEUSEC + "," \
"nodeCAcertificateFile=" + CSRBNOTEBOOKROOT + "/CSRBbin/CERTS/CA.nodes.csrb.crt," \
"nodeCertificateFile=" + CSRBNOTEBOOKROOT + "/CSRBbin/CERTS/C9BAD58F23D5A6C095C0571512CD318D.nodes.csrb.pem," \
"nodeID=" + NODEID + ",storagePath=/mnt/CSRBSTORAGE/" + NODEID + "/ " \
"-f /mnt/CSRB" + (" &" if RUNBG is True else ""))

In [ ]:
#@title Stop CSRBvfsFUSE

!pidof CSRBvfsFUSE > /dev/null && kill -9 `pidof CSRBvfsFUSE`
!umount -f {CSRBVFS} > /dev/null

# MISC

In [ ]:
!ls -l {CSRBVFS}

In [ ]:
!ps -awux | grep CSRBvfsFUSE

# CSRBappNotHotdog

## INIT

In [ ]:
#@title Clone CSRBappNotHostdogServer (Step #B1)
!git -C /content/ clone --recurse-submodules https://github.com/CSRBapp/CSRBappNotHotdogServer.git

In [ ]:
#@title Update CSRBappNotHotdogServer
!git -C /content/CSRBappNotHotdogServer pull --recurse-submodules

In [ ]:
#@title Diff CSRBappNotHotdogServer
!git -C /content/CSRBappNotHotdogServer diff --submodule=diff

## RUN Server

In [ ]:
#@title Start CSRBappNotHotdog Server (Step #B2)

os.chdir("/content/CSRBappNotHotdogServer/")
os.environ["CSRBVFS"] = CSRBVFS
%run CSRBappNotHotdog.py

## RUN Client

In [ ]:
#@title Run CSRBappNotHotdog Client (Step #B3)

os.chdir("/content/CSRBappNotHotdogServer/")
os.environ["CSRBVFS"] = CSRBVFS

if 'NODEID' not in globals():
  raise ValueError("NODEID not set. Run the NODEID cell first.")
  
SERVERID = "" #@param ["00000000000000000000000000000000"] {allow-input: true}
IMAGE = "" #@param ["/content/CSRBappNotHotdogServer/hotdog-or-not-hotdog/test/hotdog.jpg", "/content/CSRBappNotHotdogServer/hotdog-or-not-hotdog/test/bottle.jpg", "/content/CSRBappNotHotdogServer/hotdog-or-not-hotdog/test/hot2.jpg", "/content/CSRBappNotHotdogServer/hotdog-or-not-hotdog/test/wallpaper.jpg"] {allow-input: true}

%run CSRBappNotHotdogClient.py $NODEID $SERVERID $IMAGE